In [1]:
!pip install rdkit
!pip install pandas
!pip install biopython 
!pip install python-docx

In [ ]:
import pandas as pd
import random
from docx import Document
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
import os
import warnings
from collections import defaultdict
from docx.shared import Inches, Pt
from docx.oxml.ns import qn
from docx.enum.section import WD_HEADER_FOOTER
from docx.oxml import OxmlElement
from PIL import Image
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import RGBColor
from docx import Document
from docx.shared import Inches
from docx.shared import Cm


warnings.filterwarnings("ignore", category=DeprecationWarning)
random.seed(42) 


df = pd.read_csv('BBBP.csv').drop_duplicates(subset=['smiles']).reset_index(drop=True)

if len(df) < 200:
    print(f"Advertencia: Solo hay {len(df)} moléculas únicas en el dataset")
    num_moleculas = len(df)
else:
    num_moleculas = 200


moleculas_seleccionadas = random.sample(range(len(df)), num_moleculas)
moleculas_procesadas = defaultdict(int)

def generar_imagen_molecula(smiles, nombre):
    img_path = f"temp_{nombre.replace('/', '_')}.png"
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        
        AllChem.Compute2DCoords(mol)
        img = Draw.MolToImage(mol, size=(500, 500), kekulize=False, A = WD_PARAGRAPH_ALIGNMENT.CENTER)
        img.save(img_path)
        return img_path
    except Exception as e:
        print(f"Error con {nombre}: {str(e)}")
        return None

for i, idx in enumerate(moleculas_seleccionadas, 1):
    fila = df.iloc[idx]
    nombre = fila['name']
    smiles = fila['smiles']
    
    # Control de duplicados
    if smiles in moleculas_procesadas:
        continue
    moleculas_procesadas[smiles] += 1
    
    doc = Document()
    titulo = doc.add_heading(level= 0)
    titulo.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run_titulo = titulo.add_run(f'{nombre}'.upper())
    run_titulo.bold = True

    
   
    doc.add_paragraph(f"Nombre: {nombre}")
    doc.add_paragraph(f"Estructura Molecular: {smiles}")
    doc.add_paragraph(f"Clasificación: {'Positiva' if fila['p_np'] == 1 else 'Negativa'}")

    
    
    img_path = generar_imagen_molecula(smiles, nombre)
    if img_path and os.path.exists(img_path):
        doc.add_picture(img_path, width=Inches(3))
        os.remove(img_path)
    else:
        doc.add_paragraph("Estructura molecular no disponible")
    if fila['p_np'] == 1:
     doc.add_paragraph(f"La molécula {nombre}, fue encontrada, analizada y estudiada en el laboratorio, donde nos percatamos que esta molécula" 
                      "si está presente en los organismos estudiados, se llega a la conclusión que tiene una reacción positiva." )
     doc.add_paragraph('Por lo tanto, se dará inicio a la producción de esta molécula, con el fin de distribuirla en el inventario' \
     ' de la empresa "BIOENGINEERS".  ')
    else: 
        doc.add_paragraph(f"La molécula {nombre}, no fue encontrada durante las pruebas de laboratorio, se llega a la conclusión que tiene una reacción negativa.")
    
    doc.add_paragraph()
    logo = doc.add_paragraph()
    logo.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    logo.add_run().add_picture("Segundo_Parcial_Barron_Guiga_Mateo_ej2_Imagen2.png", width=Cm(2.5), height=Cm(4))
    a = doc.add_paragraph()
    a.add_run("Ingeniero Mateo Barrón\n" \
    'Jefe de empresa "BIOENGINEERS"\n' \
    'La Paz, 8 de junio de 2022')
    a.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    
    
    doc.save(f'informe_molecula{i}.docx')

[18:55:45] WARNING: not removing hydrogen atom without neighbors
[18:55:51] WARNING: not removing hydrogen atom without neighbors
[18:55:54] WARNING: not removing hydrogen atom without neighbors
[18:55:54] Explicit valence for atom # 5 N, 4, is greater than permitted
[18:56:02] WARNING: not removing hydrogen atom without neighbors
[18:56:04] WARNING: not removing hydrogen atom without neighbors
[18:56:07] WARNING: not removing hydrogen atom without neighbors
